# CSCI 3360: Course Project
This file is used to clean up the data and create a new modified file to work with.

**Project Members:** JP Park, Yiren Hou, Martha Sikora, Pragya Bhayana
**Dataset:** [Predicting Depression, Anxiety, and Stress](https://www.kaggle.com/yamqwe/depression-anxiety-stress-scales) from [Kaggle](https://www.kaggle.com) by [Yam Peleg](https://www.kaggle.com/yamqwe)

In [273]:
import pandas as pd
import numpy as np

In [274]:
df = pd.read_table("data/data.csv")

# Validity Check

In [275]:
# VCL
df = df[df[['VCL6', 'VCL9', 'VCL12']].sum(axis=1) == 0]
df = df[df[['VCL1', 'VCL4', 'VCL10', 'VCL15', 'VCL16']].sum(axis=1) > 2]

In [276]:
# testelapse + surveyelapse
elapse = df['testelapse'] + df['surveyelapse']
min_qr = elapse.quantile(0.01)
max_qr = elapse.quantile(0.99)
df = df[elapse.between(min_qr, max_qr)]

In [277]:
# age
df = df[df['age'].between(13, 80)]

# family size
df = df[df['familysize'] < 15]

# eliminate minors that voted or are/have been married
minor = df['age'] < 18
voted = df['voted'] == 1
married = df['married'] > 1
df = df[~(minor & (voted | married))]

# Calculating BIG-FIVE Personality

In [278]:
# Extraversion
df['EXT1'] = df['TIPI1']
df['EXT2'] = 8 - df['TIPI6']

# Agreeableness
df['AGR1'] = 8 - df['TIPI2']
df['AGR2'] = df['TIPI7']

# Conscientiousness
df['CON1'] = df['TIPI3']
df['CON2'] = 8 - df['TIPI8']

# Emotional Stability
df['EST1'] = 8 - df['TIPI4']
df['EST2'] = df['TIPI9']

# Openness
df['OPE1'] = df['TIPI5']
df['OPE2'] = 8 - df['TIPI10']

# Calculating DASS score and level

In [279]:
# Calculate and create columns for depression, anxiety, and stress scores
# and levels using the official DASS scale
for cat, cols, bins in \
        ('depression', '3|5|10|13|16|17|21|24|26|31|34|37|38|42', [0, 9, 13, 20, 27, np.inf]), \
        ('anxiety', '2|4|7|9|15|19|20|23|25|28|30|36|40|41', [0, 7, 9, 14, 19, np.inf]), \
        ('stress', '1|6|8|11|12|14|18|22|27|29|32|33|35|39', [0, 14, 18, 25, 33, np.inf]):
    df[f'{cat}_score'] = df.filter(regex=f"Q({cols})A").sub(1, fill_value=0).sum(axis=1)
    df[f'{cat}_level'] = pd.cut(df[f'{cat}_score'], right=False, bins=bins, labels=range(5))

# Drop unnecessary columns

In [280]:
drop = lambda regex: df.drop(df.filter(regex=regex).columns, axis=1, inplace=True)

# vocabulary check list to check validity
drop('VCL\\d+')

# Q#A - the answer given to the question
# Q#E - time spent on answering question (in milliseconds)
# Q#I - the question's position in the survey
drop('Q\\d+(A|E|I)')

# introelapse - time spent on the introduction/landing page (in seconds)
# testelapse - time spent on all the DASS questions
# surveylapse - time spent answering the rest of the demographic and survey questions
drop('.+lapse')

# TIPI - Ten Item Personality Inventory
drop('TIPI\\d+')

# Save to a file

In [281]:
df.to_csv("data/cleaned.csv", index=False)

In [282]:
df['age'].sort_values()

38134    13
27812    13
28596    13
13545    13
39230    13
         ..
10179    79
12981    80
4221     80
2217     80
10739    80
Name: age, Length: 29382, dtype: int64

In [283]:
df

,country,source,education,urban,gender,engnat,age,screensize,uniquenetworklocation,hand,...,EST1,EST2,OPE1,OPE2,depression_score,depression_level,anxiety_score,anxiety_level,stress_score,stress_level
0,IN,2,2,3,2,2,16,1,1,1,...,1,1,7,7,27,4,34,4,40,4
1,US,2,2,3,2,1,16,2,1,2,...,1,1,5,3,24,3,17,3,27,3
3,US,2,1,3,2,1,13,2,1,2,...,4,6,6,7,16,2,17,3,16,1
4,MY,2,3,2,2,2,19,2,2,3,...,2,3,5,5,32,4,40,4,29,3
5,US,2,2,3,2,2,20,2,1,1,...,7,6,7,1,13,2,6,0,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39769,US,2,1,2,2,1,16,2,1,1,...,3,5,6,1,15,2,21,4,30,3
39770,GB,2,2,2,1,1,16,2,1,1,...,3,3,6,2,36,4,12,2,15,1
39772,US,0,3,2,2,1,48,2,1,1,...,3,5,6,4,4,0,1,0,5,0
39773,US,2,3,2,2,1,20,2,1,1,...,1,3,3,4,16,2,13,2,33,4
